In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import warnings
warnings.filterwarnings("ignore")

In [2]:
!gdown --folder 18tPh3vljXUWLlfQV4GDdLlrvKJpuSRcU -q

In [3]:
data1=pd.read_csv('data/PRICES_HISTORY_DAILY.csv', sep = ';')
data2=pd.read_csv('data/TRANSACTIONS_HISTORY.csv')
data2 = data2.drop('TRANSACTION_ID', 1)
data2 = data2.drop('CUSTOMER_ID', 1)

In [4]:
tmp = pd.merge(data1, data2, how='inner', on=['PRODUCT_ID', 'DATE', 'STORE_ID']).drop_duplicates(subset=['PRODUCT_ID', 'DATE', 'STORE_ID'], keep="first")

In [5]:
tmp0=tmp
tmp0=tmp0.query('PRICE_TYPE != 0')
tmp0.to_csv('start_without_promo.csv', index=False)

In [6]:
tmp = tmp.replace({'STORE_ID':{'DC01':0, 'DC02':1}})
tmp = tmp.replace({'PRICE_TYPE':{'PROMO':0, 'REGULAR':1}})
tmp = tmp.drop(['DATE', 'PRICE_TYPE'], 1)

In [7]:
df=tmp
df.loc[df['STORE_ID']==0, 'PRODUCT_ID'] *= 10
df.loc[df['STORE_ID']==1, 'PRODUCT_ID'] *= 10
df.loc[df['STORE_ID']==1, 'PRODUCT_ID'] += 1
df_plus=df
df_plus = df_plus.assign(price_plus=df.PRICE+5)
df_plus=df_plus.drop('PRICE', 1)

In [8]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
    coef_dict

In [9]:
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
ans.to_csv('first.csv', index=False)

In [10]:
df_log = tmp0
df_log = df_log.assign(LOG_PRICE=np.log(df_log.PRICE))

In [11]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_log['PRODUCT_ID'].unique():
    df=df_log[tmp0['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
answer_log.to_csv('fourth.csv', index=False)

In [12]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp0['PRODUCT_ID'].unique():
    df=tmp0[tmp0['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
answer = answer.assign(PRODUCT_ID = answer.pr // 10 )
answer = answer.assign(STORE_ID = answer.pr % 10)
answer=answer.drop(['pr'], 1)
answer.to_csv('without_promo_first.csv', index=False) 

In [13]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_log['PRODUCT_ID'].unique():
    df=df_log[tmp0['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
answer_log.to_csv('without_promo_log_first')

In [14]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in df_plus['PRODUCT_ID'].unique():
    df1=df_plus[df_plus['PRODUCT_ID']==prod]
    X = df1['price_plus'].values.reshape(-1,1)
    y = df1['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
    coef_dict
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('plus1.csv', index=False)

In [15]:
df_plus_log = df_plus
df_plus_log = df_plus_log.assign(LOG_PRICE=np.log(df_plus_log.price_plus))

In [16]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_plus_log['PRODUCT_ID'].unique():
    df=df_plus_log[df_plus_log['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
c = np.vstack(answer_log['coef'])
answer_log['coef_b'] = c[:, 0]
answer_log['coef_k'] = c[:, 1]
answer_log = answer_log.drop('coef', axis=1)
answer_log.to_csv('plus2.csv', index=False)

In [17]:
data4 = pd.read_csv('plus2.csv', sep = ',')
data5 = pd.read_csv('plus1.csv', sep = ',')

In [18]:
data5['coef_k'].loc[data5['coef_k'] > 2.8] = -1.633449
data5.to_csv('1605(3).csv', index=False)
data5['coef_k'].loc[data5['coef_k'] > 0.000000] = -1.633449
data5.to_csv('1605(2).csv', index=False)
data5['coef_k'].loc[data5['coef_k'] >= 0.000000] = -1.633449
data5.to_csv('1605(1).csv', index=False)

In [19]:
tmp1=pd.read_csv('data/start_without_promo.csv', sep = ',')

In [20]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = Ridge(alpha = 0.2)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('Ridge_first.csv', index=False)

In [21]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = ElasticNet(normalize=True,alpha=0.001,l1_ratio=0.75)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('ElasticNet_first.csv', index=False)

In [22]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = Lasso(alpha = 0.0001)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('Lasso_first.csv', index=False)
ans

,PRODUCT_ID,STORE_ID,coef_b,coef_k
0,3046357,0,0.751724,0.128407
1,3046358,0,1.482198,-0.061359
2,3047286,0,0.896222,0.092763
3,3047287,0,1.121789,0.079844
4,3047290,0,1.446377,-0.042932
...,...,...,...,...
142,3396472,1,1.000000,0.000000
143,3396473,1,8.476413,-1.878127
144,3396474,1,38.565535,-8.889718
145,3396475,1,1.000000,0.000000
